<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Pesquisa/2_Testes_em_Massa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coordinates of an object in image

# Importando as Bibliotecas:

In [2]:
import os

# A biblioteca "os" fornece funções para interagir com o sistema operacional, como manipulação de caminhos de arquivo, 
# verificação de existência de arquivos/diretórios, criação de diretórios, etc.

import glob

# A biblioteca "glob" é usada para obter uma lista de caminhos de arquivos que correspondem a um determinado padrão. 
# É útil para buscar arquivos em um diretório com base em padrões de nomenclatura ou extensões.

import cv2

# A biblioteca "OpenCV" (Open Source Computer Vision Library) é uma biblioteca popular para processamento de imagem e visão computacional. 
# Ela fornece várias funções e algoritmos para manipulação, análise e processamento de imagens e vídeos.

import numpy as np

# A biblioteca "NumPy" é usada para operações numéricas eficientes em Python. 
# Ela fornece arrays multidimensionais (ndarrays) que são usados para armazenar e manipular dados numéricos.

import pandas as pd

# A biblioteca "Pandas" fornece estruturas de dados e funções para análise de dados. 
# Ela é amplamente utilizada para manipulação e análise de dados tabulares, como planilhas ou bancos de dados.

from google.colab.patches import cv2_imshow

# A função "cv2_imshow" é uma adaptação do OpenCV para exibir imagens diretamente no Google Colab, 
# permitindo visualizar imagens em uma célula de código sem a necessidade de salvá-las em arquivos temporários.

from google.colab import drive

# A biblioteca "drive" permite montar o Google Drive no ambiente do Google Colab, 
# facilitando o acesso a arquivos e diretórios armazenados no Google Drive.

drive.mount('/content/drive/')

# Monta o Google Drive no ambiente do Google Colab, permitindo acessar os arquivos armazenados no Google Drive.

%cd /content/drive/MyDrive/

# Altera o diretório de trabalho atual para o diretório especificado no Google Drive.

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


# Estruturas de dados:

In [5]:
nomes = [] # Armazena o nome do frame que está sendo analisado

formas = [] # Armazena as formas de cada objeto

areas = [] # Armazena a área de cada objeto
perimeters = [] # Armazena o perímetro de cada objeto

width = [] # Armazena a largura do retângulo de contorno de cada objeto
height = [] # Armazena a altura do retângulo de contorno de cada objeto
angle = [] # Armazena o ângulo (em graus) do retângulo de contorno de cada objeto
radius = [] # Armazena o raio do círculo de contorno de cada objeto

equivalent_diameter = [] # Armazena o diâmetro equivalente (em pixels) do círculo com a mesma área de cada objeto

centroids_x = [] # Armazena as coordenadas x do centróide de cada objeto
centroids_y = [] # Armazena as coordenadas y do centróide de cada objeto
aspect_ratio = [] # Armazena a relação entre largura e altura do retângulo de contorno de cada objeto
extent = [] # Armazena a razão entre a área do contorno e a área do retângulo de contorno de cada objeto
solidity = [] # Armazena a razão entre a área do contorno e a área do casco convexo de cada objeto

# Endereços:

> Lembrando que os endereços de **Entrada dos Videos** e **Destino das Imagens** estão vinculados ao drive do usuario que está rodando esse código. Caso você não seja o autor desse código os endereços deverão ser modificados.

## Pasta de Endereço dos Videos:

Lembre sempre que a extensão do video pode acabar sendo alterada. 

No meu caso tenho entradas que possuem extensão .mp4 e outras .mpg

In [16]:
# entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 15/Vídeos separados - TR e TT/grey_field_videos/*.mp4')

entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/*.mpg')

# Imprime os elementos da lista com quebras de linha entre cada elemento
print('\n'.join(entrada))

/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF504-Contexto4.mpg
/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF508-Contexto4.mpg
/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF512-Contexto4.mpg


## Pasta de Destino das Imagens:

In [7]:
#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Base de Dados/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_grey/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/'

destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/'

## Pacote de frames salvos:

Se você acertou o endereço de destino não precisa modificar a cécula abaixo:

In [29]:
frames = glob.glob(destino + '*.jpg')
print(frames)

['/content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/Dia2-Explo-CCF504-Contexto4.jpg', '/content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/Dia2-Explo-CCF508-Contexto4.jpg', '/content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/Dia2-Explo-CCF512-Contexto4.jpg']


# Detecção de Objetos:

## Extração do primeiro Frame de cada videos:

In [25]:
def extract_frames_from_videos(path_list, destino):
  for video in path_list:
    # Verificar se o caminho do vídeo existe
    if not os.path.exists(video):
      print(f"O vídeo '{video}' não existe.")
      return

    # Abrir o arquivo de vídeo
    vidcap = cv2.VideoCapture(video)

    # Verificar se o vídeo foi aberto corretamente
    if not vidcap.isOpened():
      print(f"Falha ao abrir o vídeo '{video}'.")
      return

    # Extrair o nome do vídeo a partir do caminho
    nome_completo = video.split('/')
    nome = nome_completo[9].split('.')
    nome = nome[0]

    # create a folder by the name of the video file
    if not os.path.exists("{}".format(destino)):
      os.makedirs("{}".format(destino))

    # Ler o primeiro frame
    success, image = vidcap.read()
    print('Read a new frame: ', success)

    # Verificar se houve sucesso na leitura do frame
    if not success:
      print(f"Falha ao ler o frame.")

      # Liberar os recursos utilizados pelo objeto de leitura do vídeo
      vidcap.release()
    
      break

    # Salvar o frame lido
    cv2.imwrite("{}{}.jpg".format(destino, nome), image)
    print('Creating...' + "{}{}.jpg".format(destino, nome))

    # Liberar os recursos utilizados pelo objeto de leitura do vídeo
    vidcap.release()
    # Fechar todas as janelas abertas pelo OpenCV
    cv2.destroyAllWindows()

In [24]:
extract_frames_from_videos(entrada, destino)

Read a new frame:  True
Creating.../content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/Dia2-Explo-CCF504-Contexto4.jpg
Read a new frame:  True
Creating.../content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/Dia2-Explo-CCF508-Contexto4.jpg
Read a new frame:  True
Creating.../content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/Dia2-Explo-CCF512-Contexto4.jpg


## Extração de um Frame pos Time Skip:

In [19]:
def extract_frame_from_video(video, destino, cont=1):
  # Verificar se o caminho do vídeo existe
  if not os.path.exists(video):
    print(f"O vídeo '{video}' não existe.")
    return

  # Abrir o vídeo
  vidcap = cv2.VideoCapture(video)

  # Verificar se o vídeo foi aberto corretamente
  if not vidcap.isOpened():
    print(f"Falha ao abrir o vídeo '{video}'.")
    return

  interval = 60  # intervalo em segundos (1 minuto)

  # Pular frames com base no intervalo definido
  vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(interval * cont * vidcap.get(cv2.CAP_PROP_FPS)))

  # Extrair o nome do vídeo a partir do caminho
  nome_completo = video.split('/')
  nome = nome_completo[9].split('.')
  nome = nome[0]

  # Ler o próximo frame
  success, image = vidcap.read()

  # Verificar se o frame foi lido com sucesso
  if not success:
    print(f"Falha ao ler a imagem {cont} do vídeo '{video}'.")

    # Liberar os recursos utilizados pelo objeto de leitura do vídeo
    vidcap.release()

    return

  # Salvar o frame lido
  cv2.imwrite("{}{}.jpg".format(destino, nome), image)
  print(f"Frame {cont} extraído do vídeo '{nome}' salva com sucesso em '{destino}'.")

  # Liberar os recursos utilizados pelo objeto de leitura do vídeo
  vidcap.release()
  # Fechar todas as janelas abertas pelo OpenCV
  cv2.destroyAllWindows()

## Processamento 1:

In [ ]:
# Loop que processa cada arquivo na lista de caminhos de arquivos.
for frame in frames:

  # Contador de objetos identificados
  objetos = 0

  # Função usada para ler a imagem e converter em uma matriz
  im = cv2.imread(frame)

  # Separa o nome especifico da imagem.path_list
  nome_completo = frame.split('/')
  nome = nome_completo[6].split('.')
  nome[0] 

  # Imprime as fronteiras e identifica as imagens:
  print("================================================================================")
  print("================================================================================") 
  print(nome[0])

  # Cria uma cópia da imagem original.
  img_contours = im.copy()

  # Converte a imagem para o espaço de cores em escala de cinza.
  image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  # Converte a imagem para o tipo float32.
  image32f = np.float32(image)

  # Aplica um filtro de média à imagem.
  mu = cv2.blur(image32f, (3, 3))

  # Aplica um filtro de variância à imagem.
  mu2 = cv2.blur(image32f*image32f, (3, 3))
  sigma = cv2.sqrt(mu2 - mu*mu)
  sigma = sigma * 10
  sigma = sigma.astype("uint8")

  # Aplica um filtro de borramento com kernel de 5x5
  sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

  # Aplicando o filtro Laplaciano
  laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

  # Normalizando a saída para o intervalo de 0 a 255
  sigma = cv2.convertScaleAbs(laplacian)

  # Aplica um filtro de Sobel em x e y
  sobelx = cv2.Sobel(sigma, cv2.CV_64F, 1, 0, ksize=3)
  sobely = cv2.Sobel(sigma, cv2.CV_64F, 0, 1, ksize=3)

  # Calcula a magnitude do gradiente
  gradient_magnitude = cv2.magnitude(sobelx, sobely)

  # Normaliza a magnitude do gradiente para o intervalo de 0 a 255
  gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  # Aplica uma limiarização com método de Otsu à imagem do gradiente
  _, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  # Define o kernel para dilatação e erosão
  kernel = np.ones((5,5), np.uint8)

  # Dilata a imagem limiarizada
  gradient_threshold = cv2.dilate(gradient_threshold, kernel, iterations=2)

  # Encontra os contornos na imagem limiarizada.
  contours, hierarchy = cv2.findContours(gradient_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  # Loop sobre os contornos
  for cnt in contours:

    # Aproxima o contorno por uma sequência de pontos
    approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

    # Calcula a área e o perímetro do contorno
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    # Verifica se o contorno é um círculo
    if perimeter > 0:
      circularity = 4 * np.pi * area / (perimeter ** 2)
    else:
      circularity = 0

    # Desenha um círculo
    if circularity > 0.30:

      forma = "circulo"

      (x, y), radius = cv2.minEnclosingCircle(cnt)

      # Limita o raio máximo do círculo
      max_radius = 100  # Define o raio máximo desejado
      radius = min(radius, max_radius)  # Limita o raio ao valor máximo

      center = (int(x), int(y))
      radius = int(radius)

      #if area > 2000 or circularity > 53:
      if area > 2000:
        if radius < 100 and radius > 20:
          cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          print(" -> circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          objetos = objetos + 1
      else:
        print("circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))

    # Desenha um quadrado
    else:
      rect = cv2.minAreaRect(cnt)
      
      forma = "quadrado"
      radius = 0

      (x, y), (w, h), angle = rect

      if min(w, h) == 0:
        aspect_ratio = 0
      else:
        aspect_ratio = max(w, h) / min(w, h)
      

      if aspect_ratio < 2:
      #if aspect_ratio < 3:  
        # Desenha um quadrado
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        if area > 2000 and area < 6543.5:
          cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          print(" -> circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          objetos = objetos + 1
      else:
          print("circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))

  # Exibe a imagem pos-filtro.
  #cv2_imshow(sigma)
  # Exibe a imagem limiarizada.
  #cv2_imshow(gradient_threshold)
  # Exibe a imagem original copiada com os contornos encontrados.
  cv2_imshow(img_contours)
  print("Foram encontrados: " + str(objetos) + " Objetos") 
  print()
  print()

## Processamento 2:

In [ ]:
# Loop que processa cada arquivo na lista de caminhos de arquivos.
for frame in frames:

  objetos = 0

  # The function cv2.imread() is used to read an image.
  im = cv2.imread(frame)

  # Separa o nome especifico da imagem.
  nome_completo = frame.split('/')
  nome = nome_completo[6].split('.')
  nome[0]

  # Imprime as fronteiras e identifica as imagens:
  print("================================================================================")
  print("================================================================================") 
  print(nome[0])

  # Cria uma cópia da imagem original.
  img_contours = im.copy()

  # Converte a imagem para o espaço de cores em escala de cinza.
  image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  # Converte a imagem para o tipo float32.
  image32f = np.float32(image)

  # Aplica um filtro de média à imagem.
  mu = cv2.blur(image32f, (3, 3))

  # Aplica um filtro de variância à imagem.
  mu2 = cv2.blur(image32f*image32f, (3, 3))
  sigma = cv2.sqrt(mu2 - mu*mu)
  sigma = sigma * 10
  sigma = sigma.astype("uint8")

  # Aplica um filtro de borramento com kernel de 5x5
  sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

  # Aplicando o filtro Laplaciano
  laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

  # Normalizando a saída para o intervalo de 0 a 255
  sigma = cv2.convertScaleAbs(laplacian)

  # Aplica um filtro de Sobel em x e y
  sobelx = cv2.Sobel(sigma, cv2.CV_64F, 1, 0, ksize=3)
  sobely = cv2.Sobel(sigma, cv2.CV_64F, 0, 1, ksize=3)

  # Calcula a magnitude do gradiente
  gradient_magnitude = cv2.magnitude(sobelx, sobely)

  # Normaliza a magnitude do gradiente para o intervalo de 0 a 255
  gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  # Aplica uma limiarização com método de Otsu à imagem do gradiente
  _, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  # Define o kernel para dilatação e erosão
  kernel = np.ones((5,5), np.uint8)

  # Dilata a imagem limiarizada
  gradient_threshold = cv2.dilate(gradient_threshold, kernel, iterations=2)

  # Encontra os contornos na imagem limiarizada.
  contours, hierarchy = cv2.findContours(gradient_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  # Loop sobre os contornos
  for cnt in contours:

    # Aproxima o contorno por uma sequência de pontos
    approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

    # Calcula a área e o perímetro do contorno
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    # Verifica se o contorno é um círculo
    if perimeter > 0:
      circularity = 4 * np.pi * area / (perimeter ** 2)
    else:
      circularity = 0

    # Desenha um círculo
    if circularity > 0.30:

      forma = "circulo"

      (x, y), radius = cv2.minEnclosingCircle(cnt)

      # Limita o raio máximo do círculo
      max_radius = 100  # Define o raio máximo desejado
      radius = min(radius, max_radius)  # Limita o raio ao valor máximo

      center = (int(x), int(y))
      radius = int(radius)

      #if area > 2000 or circularity > 53:
      if area > 2000:
        if radius < 100 and radius > 20:
          cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          print(" -> circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          objetos = objetos + 1
      else:
        print("circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))

    # Desenha um quadrado
    else:
      rect = cv2.minAreaRect(cnt)
      
      forma = "quadrado"
      radius = 0

      (x, y), (w, h), angle = rect

      if min(w, h) == 0:
        aspect_ratio = 0
      else:
        aspect_ratio = max(w, h) / min(w, h)
      

      if aspect_ratio < 2:
      #if aspect_ratio < 3:  
        # Desenha um quadrado
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        if area > 2000 and area < 6543.5:
          cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          print(" -> circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))
          print("================================================================================================================================================================")
          print("================================================================================================================================================================") 
          objetos = objetos + 1
      else:
          print("circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))

  # Exibe a imagem pos-filtro.
  #cv2_imshow(sigma)
  # Exibe a imagem limiarizada.
  #cv2_imshow(gradient_threshold)
  # Exibe a imagem original copiada com os contornos encontrados.
  cv2_imshow(img_contours)
  print("Foram encontrados: " + str(objetos) + " Objetos") 
  print()
  print()